# 🚗 YOLOv8 Pothole & Crack Detection — Training on Colab

This notebook trains YOLOv8 for pothole and crack detection using a **free T4 GPU** on Google Colab.

## Steps:
1. Connect to GPU runtime
2. Clone the GitHub repo
3. Install dependencies
4. Download datasets from Roboflow
5. Train (2-stage transfer learning)
6. Download trained model

---

> ⚠️ **First**: Go to **Runtime → Change runtime type → T4 GPU** before running any cells.

## 1. Verify GPU

In [1]:
!nvidia-smi
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

Thu Feb 12 12:17:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

AttributeError: 'torch._C._CudaDeviceProperties' object has no attribute 'total_mem'

## 2. Clone Repository & Install Dependencies

In [2]:
import os

# Clone the repo
REPO_URL = "https://github.com/Shubhamf0073/Margdrashti_models.git"
REPO_DIR = "/content/Margdrashti_models"

if os.path.exists(REPO_DIR):
    print(f"Repo already cloned at {REPO_DIR}, pulling latest...")
    !cd {REPO_DIR} && git pull
else:
    !git clone {REPO_URL} {REPO_DIR}

os.chdir(REPO_DIR)
print(f"\nWorking directory: {os.getcwd()}")
!ls -la

Cloning into '/content/Margdrashti_models'...
remote: Enumerating objects: 30058, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30058 (delta 5), reused 25 (delta 5), pack-reused 30030 (from 1)
Receiving objects: 100% (30058/30058), 459.01 MiB | 30.13 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updating files: 100% (67275/67275), done.

Working directory: /content/Margdrashti_models
total 196
drwxr-xr-x 8 root root  4096 Feb 12 12:18 .
drwxr-xr-x 1 root root  4096 Feb 12 12:18 ..
drwxr-xr-x 4 root root  4096 Feb 12 12:18 data
-rw-r--r-- 1 root root 11277 Feb 12 12:18 deploy_inference_realtime_fast.py
-rw-r--r-- 1 root root 11535 Feb 12 12:18 deploy_inference_realtime_fast_throttled.py
-rw-r--r-- 1 root root  9582 Feb 12 12:18 deploy_inference_realtime.py
-rw-r--r-- 1 root root 31325 Feb 12 12:18 deploy_inference_smooth.py
-rw-r--r-- 1 root root 15201 Feb 12 12:18 deploy_inference_yolov8.py
-rw-r--r-- 1 root root

In [3]:
# Install required packages
!pip install -q ultralytics roboflow pyyaml

# Verify installation
import ultralytics
print(f"Ultralytics: {ultralytics.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 137.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.4.14


## 3. Download Dataset from Roboflow

In [4]:
# ============================================================
# 🔑 SET YOUR ROBOFLOW API KEY HERE
# ============================================================
ROBOFLOW_KEY = "lcZrl8Z8hFNZYgrdN33C"  # <-- Replace if needed

!python scripts/yolov8_detection/download_datasets.py \
    --roboflow_key {ROBOFLOW_KEY} \
    --output_dir data/pothole_crack_detection \
    --skip_kaggle

YOLOV8 DATASET PREPARATION
Output directory: data/pothole_crack_detection
Temporary downloads: data/pothole_crack_detection/temp_downloads

DOWNLOADING ROBOFLOW DATASETS

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to data/pothole_crack_detection/temp_downloads/GeraPotHole in yolov8:: 100% 2866/2866 [00:00<00:00, 4640.48it/s]
✓ Downloaded GeraPotHole to /content/Margdrashti_models/data/pothole_crack_detection/temp_downloads/GeraPotHole

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to data/pothole_crack_detection/temp_downloads/Kartik in yolov8:: 100% 2908/2908 [00:00<00:00, 5980.04it/s]
✓ Downloaded Kartik to /content/Margdrashti_models/data/pothole_crack_detection/temp_downloads/Kartik

MERGING DATASETS

Processing GeraPotHole...

Processing Kartik...

Total images collected: 2875

Copying 2012 images to train split...

Copying 575 images to val split...

Copying 288 images to test split...

✓ D

In [5]:
# Verify dataset
!cat data/pothole_crack_detection/data.yaml
print()
!echo "Train images:" && ls data/pothole_crack_detection/images/train/ | wc -l
!echo "Val images:"   && ls data/pothole_crack_detection/images/val/ | wc -l
!echo "Test images:"  && ls data/pothole_crack_detection/images/test/ | wc -l

names:
- pothole
- crack
- longitudinal_crack
- transverse_crack
nc: 4
path: /content/Margdrashti_models/data/pothole_crack_detection
test: images/test
train: images/train
val: images/val

Train images:
2012
Val images:
575
Test images:
288


## 4. Fix data.yaml Path for Colab

The data.yaml has an absolute path from local machine. We need to update it for Colab.

In [6]:
import yaml

yaml_path = "data/pothole_crack_detection/data.yaml"

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update to Colab absolute path
data['path'] = f"{REPO_DIR}/data/pothole_crack_detection"

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("Updated data.yaml:")
!cat {yaml_path}

Updated data.yaml:
names:
- pothole
- crack
- longitudinal_crack
- transverse_crack
nc: 4
path: /content/Margdrashti_models/data/pothole_crack_detection
test: images/test
train: images/train
val: images/val


## 5. Train YOLOv8 (Two-Stage Transfer Learning)

- **Stage 1** (15 epochs): Backbone frozen, trains detection head only
- **Stage 2** (30 epochs): Full fine-tuning with lower learning rate

On a T4 GPU this should take **~20-40 minutes** total.

In [7]:
!python scripts/yolov8_detection/train_yolov8.py \
    --data data/pothole_crack_detection/data.yaml \
    --model yolov8n.pt \
    --epochs_stage1 15 \
    --epochs_stage2 30 \
    --batch 32 \
    --device 0 \
    --cache


                 YOLOV8 TWO-STAGE TRAINING                  

Start time: 2026-02-12 12:19:16

Configuration:
  Data: data/pothole_crack_detection/data.yaml
  Model: yolov8n.pt
  Batch size: 32
  Image size: 640
  Device: 0
  Output: scripts/runs

✓ GPU detected: Tesla T4 (14.6 GB)

                     DATASET VALIDATION                     
Dataset: data/pothole_crack_detection/data.yaml
Classes (4): ['pothole', 'crack', 'longitudinal_crack', 'transverse_crack']
Train: 2012 images at /content/Margdrashti_models/data/pothole_crack_detection/images/train
Val  : 575 images at /content/Margdrashti_models/data/pothole_crack_detection/images/val
Test : 288 images at /content/Margdrashti_models/data/pothole_crack_detection/images/test

             STAGE 1: FROZEN BACKBONE TRAINING              

Initializing YOLOv8 from yolov8n.pt...

Freezing first 10 layers...
Frozen parameters: 3,157,200
Trainable parameters: 0

Training configuration:
  Epochs: 15
  Batch size: 32
  Image size: 640
  

In [8]:
!cd /content/Margdrashti_models && git pull origin master

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 589 bytes | 147.00 KiB/s, done.
From https://github.com/Shubhamf0073/Margdrashti_models
 * branch              master     -> FETCH_HEAD
   3a66c5e9..33f9a694  master     -> origin/master
Updating 3a66c5e9..33f9a694
Fast-forward
 scripts/yolov8_detection/train_yolov8.py | 6 ++++--
 1 file changed, 4 insertions(+), 2 deletions(-)


In [9]:
!python scripts/yolov8_detection/train_yolov8.py \
    --data data/pothole_crack_detection/data.yaml \
    --model yolov8n.pt \
    --epochs_stage2 30 \
    --batch 32 \
    --device 0 \
    --cache \
    --skip_stage1 \
    --resume_stage1 runs/detect/scripts/runs/yolov8n_stage1/weights/best.pt


                 YOLOV8 TWO-STAGE TRAINING                  

Start time: 2026-02-12 12:34:40

Configuration:
  Data: data/pothole_crack_detection/data.yaml
  Model: yolov8n.pt
  Batch size: 32
  Image size: 640
  Device: 0
  Output: scripts/runs

✓ GPU detected: Tesla T4 (14.6 GB)

                     DATASET VALIDATION                     
Dataset: data/pothole_crack_detection/data.yaml
Classes (4): ['pothole', 'crack', 'longitudinal_crack', 'transverse_crack']
Train: 2012 images at /content/Margdrashti_models/data/pothole_crack_detection/images/train
Val  : 575 images at /content/Margdrashti_models/data/pothole_crack_detection/images/val
Test : 288 images at /content/Margdrashti_models/data/pothole_crack_detection/images/test

Skipping Stage 1, using checkpoint: runs/detect/scripts/runs/yolov8n_stage1/weights/best.pt

                 STAGE 2: FULL FINE-TUNING                  

Loading model from Stage 1: runs/detect/scripts/runs/yolov8n_stage1/weights/best.pt...

Unfreezing all 

## 6. View Training Results

In [10]:
from IPython.display import Image, display
import glob

# Show training curves
for stage in ['yolov8n_stage1', 'yolov8n_stage2']:
    results_img = f"scripts/runs/{stage}/results.png"
    if os.path.exists(results_img):
        print(f"\n{'='*50}")
        print(f"{stage} Results")
        print(f"{'='*50}")
        display(Image(filename=results_img, width=800))

# Show confusion matrix if available
for stage in ['yolov8n_stage1', 'yolov8n_stage2']:
    cm_img = f"scripts/runs/{stage}/confusion_matrix.png"
    if os.path.exists(cm_img):
        print(f"\n{stage} Confusion Matrix")
        display(Image(filename=cm_img, width=600))

## 7. Download Trained Model

Download the best model weights to your local machine.

In [11]:
from google.colab import files
import shutil

# Path to the best model from Stage 2
best_model = "scripts/runs/yolov8n_stage2/weights/best.pt"

if os.path.exists(best_model):
    print(f"Best model: {best_model}")
    print(f"Size: {os.path.getsize(best_model) / 1024 / 1024:.1f} MB")

    # Also zip the entire runs directory for full results
    shutil.make_archive("/content/training_results", 'zip', "scripts/runs")

    print("\n📥 Downloading best model weight...")
    files.download(best_model)

    print("\n📥 Downloading full results archive...")
    files.download("/content/training_results.zip")
else:
    # Fallback to Stage 1 model
    fallback = "scripts/runs/yolov8n_stage1/weights/best.pt"
    if os.path.exists(fallback):
        print(f"Stage 2 not found, downloading Stage 1 model: {fallback}")
        files.download(fallback)
    else:
        print("❌ No trained model found. Check training output above for errors.")

❌ No trained model found. Check training output above for errors.


## 8. (Optional) Test Inference on Colab

Quick test on a sample image from the test set.

In [ ]:
from ultralytics import YOLO
from IPython.display import Image, display

# Load trained model
model = YOLO("scripts/runs/yolov8n_stage2/weights/best.pt")

# Get a test image
test_images = glob.glob("data/pothole_crack_detection/images/test/*")
if test_images:
    results = model.predict(
        source=test_images[:5],
        conf=0.5,
        save=True,
        project="test_predictions",
        name="samples",
        exist_ok=True
    )

    # Display predictions
    for img_path in glob.glob("test_predictions/samples/*"):
        display(Image(filename=img_path, width=600))
else:
    print("No test images found")

---
## ✅ Next Steps (on your local machine)

After downloading `best.pt`, place it at `scripts/runs/yolov8n_stage2/weights/best.pt` in your local repo and run:

```bash
# On a video file
python deploy_inference_yolov8.py \
    --model scripts/runs/yolov8n_stage2/weights/best.pt \
    --roi roi_highway_shorter.json \
    --source path/to/your/video.mp4 \
    --confidence 0.5 \
    --save_output results/output.mp4

# On webcam
python deploy_inference_yolov8.py \
    --model scripts/runs/yolov8n_stage2/weights/best.pt \
    --roi roi_highway_shorter.json \
    --source 0
```